## Parallel AI Inference on image dataset retrieved from a cloud
This notebook is an example how to use DeGirum PySDK to do multi-threaded AI inference of an image dataset.
Image dataset is retrieved from the cloud using `fiftyone` API.

#### This sample uses the following external packages, which need to be installed:
1. **fiftyone**: `pip install fiftyone`

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

#### Specify dataset parameters here

In [ ]:
# specify the name of desired dataset to retrieve;
# see https://voxel51.com/docs/fiftyone/user_guide/dataset_zoo/datasets.html
foz_dataset_name = "coco-2017"

# specify list of class labels to retrieve; None for all classes
foz_classes = None # ["car", "cup", "person"]

# specify which splits to download ("train", "validation", "test")
foz_splits = "validation"

# specify # of samples in dataset to retrieve
samples_num = 1000

# specify the model to be used for inference
model_name = "yolo_v5s_coco--576x576_quant_n2x_orca_1"

# specify number of parallel threads to run inference
nthreads = 3

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, mytools

cloud_token = mytools.get_token() # get cloud API access token from env.ini file
cloud_zoo_url = mytools.get_cloud_zoo_url() # get cloud zoo URL from env.ini file

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

# 1. Inference on the DeGirum Cloud Platform
zoo = dg.connect(dg.CLOUD, cloud_zoo_url, cloud_token)

# 2. Inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN
# zoo = dg.connect(mytools.get_ai_server_hostname(), cloud_zoo_url, cloud_token)

# 3. Inference on DeGirum ORCA accelerator installed on your computer
# zoo = dg.connect(dg.LOCAL, cloud_zoo_url, cloud_token)

#### The rest of the cells below should run without any modifications

In [ ]:
import threading, queue, sys

fo = mytools.import_fiftyone() # import 'fiftyone' package for dataset management

In [ ]:
# download dataset
dataset = fo.zoo.load_zoo_dataset(
    foz_dataset_name,
    dataset_dir="./my-dataset", 
    classes=foz_classes,
    split=foz_splits,
    max_samples=samples_num,
    shuffle=True,
    drop_existing_dataset=False)

In [ ]:
# define worker thread function:
# it takes frames from `inqueue`, performs ML inference, and puts results into `outqueue`
def worker(inqueue, outqueue):
        
    # load model
    model = zoo.load_model(model_name)
    model.output_confidence_threshold = 0.1
    myframes = []

    # define frame source generator function:
    # it gets frames from input queue and yields frame filenames one at a time
    def source():
        while True:
            frame = inqueue.get()
            if frame is None:
                # poison pill received: return poison pill back to queue to stop other threads
                inqueue.put(None)
                break
            myframes.append(frame)
            yield frame.filepath

    # do batch prediction on frame source
    for res in model.predict_batch(source()):
        # put a tuple of inference result and input frame into output queue
        outqueue.put((res, myframes.pop(0)))

In [ ]:
# create inference task and result queues
task_queue = queue.Queue()
result_queue = queue.Queue()

# fill task queue with data from dataset
nframes = dataset.count()
print(f"Queueing {nframes} frames...")
for frame in dataset:
    task_queue.put(frame)

task_queue.put(None) # poison pill is last

# run ML inference in threads
print(f"Running inference...")
tmr = mytools.Timer()
threads = []
for n in range(nthreads): # start worker threads
    t = threading.Thread(target=worker, args=(task_queue, result_queue))
    threads.append(t)
    t.start()

for t in threads: # wait for completion
    t.join()

elapsed_s = tmr()
print(f"Inference done: {nframes} frames in {elapsed_s:.1f} s = {nframes/elapsed_s:.1f} FPS")

# process result queue: add inference results to dataset
print(f"Processing results...")
for n in range(nframes):
    res, frame = result_queue.get()
    w, h = res.image.size
    detections = []
    for box in res.results:
        # Convert to [top-left-x, top-left-y, width, height]
        # in relative coordinates in [0, 1] x [0, 1]
        x1, y1, x2, y2 = box["bbox"]
        rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]
        detections.append(fo.Detection(label=box["label"], bounding_box=rel_box, confidence=box["score"]))
    frame["predictions"] = fo.Detections(detections=detections)
    frame.save()

print(f"Done!")

In [ ]:
# run evaluation on predictions
eval_result = dataset.evaluate_detections("predictions", compute_mAP=True)

In [ ]:
# print some evaluation results
print( f"mAP = {eval_result.mAP():.2f}\n")
eval_result.print_report(classes=foz_classes)